In [36]:
import pandas as pd
import os
from google.cloud import storage
from urllib.parse import urlparse, unquote
from tqdm import tqdm
import PyPDF2

In [27]:
data = pd.read_csv('./datasets/resumes.csv')
data.head()

,url
0,https://storage.cloud.google.com/mercor_dashbo...
1,https://storage.cloud.google.com/mercor_dashbo...
2,https://storage.cloud.google.com/mercor_dashbo...
3,https://storage.cloud.google.com/mercor_dashbo...
4,https://storage.cloud.google.com/mercor_dashbo...


In [28]:
urls = data['url'].tolist()
urls[:5]

['https://storage.cloud.google.com/mercor_dashboard_data/aks76625%40gmail.com_resume.pdf',
 'https://storage.cloud.google.com/mercor_dashboard_data/sharanlata132%40gmail.com_resume.pdf',
 'https://storage.cloud.google.com/mercor_dashboard_data/sjtailor27%40gmail.com_resume.pdf',
 'https://storage.cloud.google.com/mercor_dashboard_data/deveshnegi27%40gmail.com_resume.pdf',
 'https://storage.cloud.google.com/mercor_dashboard_data/sumeetghumare4%40gmail.com_resume.pdf']

In [29]:
filenames = [unquote(url.split("/")[-1]).split('.pdf')[0]+'.pdf' for url in urls]

filenames[:5]

['aks76625@gmail.com_resume.pdf',
 'sharanlata132@gmail.com_resume.pdf',
 'sjtailor27@gmail.com_resume.pdf',
 'deveshnegi27@gmail.com_resume.pdf',
 'sumeetghumare4@gmail.com_resume.pdf']

In [30]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"../Mercor Dashboard IAM Admin (5).json"

In [31]:
def download_cs_file(bucket_name, file_name, destination_file_name): 
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(file_name)
    blob.download_to_filename(destination_file_name)

    return True

In [32]:
filenames[253]

'sandhyalakshmi676@gmail.com_resume.pdf'

In [34]:
bucket_name = "mercor_dashboard_data"
destination_file_name = "./resume_data/"
error_files_indices = []
for idx, filename in tqdm(enumerate(filenames), total=len(filenames)):
    try:
        download_cs_file(bucket_name, filename, destination_file_name + filename)
    except Exception as e:
        print(e)
        error_files_indices.append(idx)

 25%|██▌       | 254/1000 [07:11<17:51,  1.44s/it]  

404 GET https://storage.googleapis.com/download/storage/v1/b/mercor_dashboard_data/o/sandhyalakshmi676%40gmail.com_resume.pdf?alt=media: No such object: mercor_dashboard_data/sandhyalakshmi676@gmail.com_resume.pdf: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)


100%|██████████| 1000/1000 [53:53<00:00,  3.23s/it]   


In [35]:
os.listdir('./resume_data/')

['adityanooka@gmail.com_resume.pdf',
 '0shivam33@gmail.com_resume.pdf',
 'kumarsingh.rohit7@gmail.com_resume.pdf',
 'vireshdev5e@gmail.com_resume.pdf',
 'aryan672002@gmail.com_resume.pdf',
 'hammicralphie@gmail.com_resume.pdf',
 'f20201026@goa.bits-pilani.ac.in_resume.pdf',
 'mohanka2102@gmail.com_resume.pdf',
 'diddigeethakrishna@gmail.com_resume.pdf',
 'nancyrupani1994@gmail.com_resume.pdf',
 'parthvik2000@gmail.com_resume.pdf',
 'ayushwanjari18@gmail.com_resume.pdf',
 'harsh.sharma1598@gmail.com_resume.pdf',
 'nayanpattanayak2014@gmail.com_resume.pdf',
 'mailtoadhil@gmail.com_resume.pdf',
 'phmoraesrodrigues@gmail.com_resume.pdf',
 'snehilvijayvargiya8959@gmail.com_resume.pdf',
 'faldu3654@gmail.com_resume.pdf',
 'parthprabhune12@gmail.com_resume.pdf',
 'sravankumar.mode@gmail.com_resume.pdf',
 'savanthsushmitha@gmail.com_resume.pdf',
 'aniketgupta0324@gmail.com_resume.pdf',
 'deotalechinmay06@gmail.com_resume.pdf',
 'irm2017005@iiita.ac.in_resume.pdf',
 'anujnb2112@gmail.com_resume

In [37]:
error_files_indices

[253]

# Extracting text from pdf resumes

In [47]:
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = []
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text.append(page.extract_text())
        return "\n".join(text)
    
def extract_resume_text(file_path):
    if file_path.endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format")
    
resume_texts = []
error_resumes = []
for file_name in tqdm(os.listdir('resume_data')):
    file_path = os.path.join('./resume_data/', file_name)
    try:
        text = extract_resume_text(file_path)
    except Exception as e:
        print(e)
        error_resumes.append(file_path)
    resume_texts.append(text)

 66%|██████▌   | 662/1000 [01:03<00:24, 13.80it/s]

Cannot read an empty file


100%|██████████| 1000/1000 [01:35<00:00, 10.50it/s]


In [49]:
resume_texts[0]

'ADITYA NOOKA\nHyderabad, India\n+91 98490 79490 adityanooka@gmail.com\nPortfolio LinkedIn GitHub Medium\nEDUCATION\nVellore Institute of Technology, Vellore September 2020 - July 2024\nBachelor of Technology in Computer Science and Engineering\nGPA: 7.88 (till Semester 7)\nSri Chaitanya Junior Kalasala, Hyderabad June 2015 - March 2020\nGrade XII Percentage: 91.50\nGrade X CGPA: 9.5\nEXPERIENCE\nupGrad January 2024 - Present\nUI/UX Design Bootcamp\n•Undergoing UI/UX design training for hands-on expertise and industry-relevant knowledge.\nSmartInternz May 2023 - July 2023\nSalesforce Administrator - Internship\n•Completed technical training and earned the Salesforce Administrator Superbadge.\n•Showcased proficiency in using Administrative Tools, and developed efficient CRM solutions.\nPROJECTS\nCAB Co. (Personal Project) Figma Link\n•Designed a user-friendly Figma interface for taxi booking, featuring clear controls and a simplified layout.\n•Enhanced the booking experience by implemen